In [202]:
import pandas as pd

In [203]:
import numpy as np

## 1. Data Loadind

In [204]:
matches = pd.read_csv("data/match-results.9.csv")

In [205]:
matches.head(2)

,Unnamed: 0,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,0,brasileiro-a,2018,9,04-06,Paraná,Fluminense,2,1,MANDANTE
1,1,brasileiro-a,2018,9,03-06,Ceará,Cruzeiro,0,1,VISITANTE


In [206]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6214 entries, 0 to 6213
Data columns (total 10 columns):
Unnamed: 0        6214 non-null int64
campeonato        6214 non-null object
ano               6214 non-null int64
rodada            6214 non-null int64
data              6214 non-null object
mandante          6214 non-null object
visitante         6214 non-null object
gols_mandante     6214 non-null int64
gols_visitante    6214 non-null int64
resultado         6214 non-null object
dtypes: int64(5), object(5)
memory usage: 485.5+ KB


In [207]:
stars = pd.read_csv("data/team_stars.csv", ';')

In [208]:
stars.head(2)

,time,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols,forca_casa,forca_fora,forca
0,São Paulo,598,10,10,10,10,10,10,10.00,10.00,10.0
1,Corinthians,560,9,10,10,6,9,8,8.25,9.75,9.5


In [209]:
stars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 11 columns):
time               41 non-null object
jogos_total        41 non-null int64
aprov_casa         41 non-null int64
aprov_fora         41 non-null int64
aprov_total        41 non-null int64
media_gols_casa    41 non-null int64
media_gols_fora    41 non-null int64
media_gols         41 non-null int64
forca_casa         41 non-null float64
forca_fora         41 non-null float64
forca              41 non-null float64
dtypes: float64(3), int64(7), object(1)
memory usage: 3.6+ KB


In [210]:
matches.groupby(["ano", "resultado"])[["resultado"]].count()

resultado
ano  resultado           
2003 EMPATE           142
     MANDANTE         297
     VISITANTE        113
2004 EMPATE           140
     MANDANTE         288
     VISITANTE        124
2005 EMPATE           103
     MANDANTE         235
     VISITANTE        124
2006 EMPATE            97
     MANDANTE         191
     VISITANTE         92
2007 EMPATE            90
     MANDANTE         192
     VISITANTE         98
2008 EMPATE            96
     MANDANTE         208
     VISITANTE         76
2009 EMPATE           102
     MANDANTE         195
     VISITANTE         83
2010 EMPATE           118
     MANDANTE         179
     VISITANTE         83
2011 EMPATE           105
     MANDANTE         184
     VISITANTE         91
2012 EMPATE           105
     MANDANTE         183
     VISITANTE         92
2013 EMPATE           108
     MANDANTE         184
     VISITANTE         88
2014 EMPATE            92
     MANDANTE         197
     VISITANTE         91
2015 EMPATE            91
     MANDANTE         200
     VISITANTE         89
2016 EMPATE            95
     MANDANTE         202
     VISITANTE         83
2017 EMPATE           103
     MANDANTE         167
     VISITANTE        110
2018 EMPATE            24
     MANDANTE          47
     VISITANTE         17

In [211]:
matches.groupby(["resultado"])[["ano"]].count()

,ano
resultado,
EMPATE,1611
MANDANTE,3149
VISITANTE,1454


### Totais por resultado

In [212]:
totais_resultados = matches.groupby("resultado").size()

In [213]:
total_jogos = totais_resultados.sum()

In [214]:
resultados_DF = pd.DataFrame({'resultados': totais_resultados.keys(), 'totais': totais_resultados.values})

In [215]:
resultados_DF

,resultados,totais
0,EMPATE,1611
1,MANDANTE,3149
2,VISITANTE,1454


In [216]:
def percentual(valor, total):
    return round(valor/total, 4)

In [217]:
resultados_DF['%'] = resultados_DF.apply (lambda row: percentual(row["totais"], total_jogos),axis=1)

In [218]:
resultados_DF

,resultados,totais,%
0,EMPATE,1611,0.2593
1,MANDANTE,3149,0.5068
2,VISITANTE,1454,0.2340


In [219]:
seasons = list(jogos["ano"].unique())

In [220]:
seasons

[2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003]

In [221]:
def classificar(jogos, rodada, classificacao):
    
    classificacao_nova = classificacao.copy()
    
    for jogo in jogos[jogos["rodada"] == rodada].iterrows():
        jogo_info = jogo[1];
        if (jogo_info["resultado"] == "MANDANTE"):            
            classificacao_nova.loc[jogo_info["mandante"]]["pts"] = classificacao.loc[jogo_info["mandante"]]["pts"] + 3
            classificacao_nova.loc[jogo_info["mandante"]]["v"] = classificacao.loc[jogo_info["mandante"]]["v"] + 1            
        elif (jogo_info["resultado"] == "VISITANTE"):
            classificacao_nova.loc[jogo_info["visitante"]]["pts"] = classificacao.loc[jogo_info["visitante"]]["pts"] + 3
            classificacao_nova.loc[jogo_info["visitante"]]["v"] = classificacao.loc[jogo_info["visitante"]]["v"] + 1
        elif (jogo_info["resultado"] == "EMPATE"):     
            classificacao_nova.loc[jogo_info["mandante"]]["pts"] = classificacao.loc[jogo_info["mandante"]]["pts"] + 1
            classificacao_nova.loc[jogo_info["visitante"]]["pts"] = classificacao.loc[jogo_info["visitante"]]["pts"] + 1
            
        classificacao_nova.loc[jogo_info["mandante"]]["gp"] = classificacao.loc[jogo_info["mandante"]]["gp"] + jogo_info["gols_mandante"]    
        classificacao_nova.loc[jogo_info["mandante"]]["gc"] = classificacao.loc[jogo_info["mandante"]]["gc"] + jogo_info["gols_visitante"]
        
        classificacao_nova.loc[jogo_info["visitante"]]["gp"] = classificacao.loc[jogo_info["visitante"]]["gp"] + jogo_info["gols_visitante"]    
        classificacao_nova.loc[jogo_info["visitante"]]["gc"] = classificacao.loc[jogo_info["visitante"]]["gc"] + jogo_info["gols_mandante"]    
        
        classificacao_nova.loc[jogo_info["mandante"]]["jogos"] = classificacao.loc[jogo_info["mandante"]]["jogos"] + 1    
        classificacao_nova.loc[jogo_info["visitante"]]["jogos"] = classificacao.loc[jogo_info["visitante"]]["jogos"] + 1
    
    classificacao_nova.eval("sg = gp - gc", inplace=True)
    classificacao_nova.sort_values(["pts", "v", "sg", "gp"], ascending=[0,0,0,0], inplace=True)
        
    for i in range(1, len(classificacao_nova.index.values.tolist())+1):   
        classificacao_nova.loc[classificacao_nova.index[i-1]]["posicao"] = i
    
    return classificacao_nova

In [222]:
ranking_DF = {}

In [223]:
for season in seasons:         
    
    turns = list(matches.query("ano == " + str(season)).sort_values("rodada")["rodada"].unique())    
    teams = list(matches.query("ano == " + str(season))["mandante"].unique())
 
    ranking_DF[season] = {}
    
    ranking = pd.DataFrame({"time":teams, "posicao": np.zeros(len(teams)), "jogos": np.zeros(len(teams)), "pts": np.zeros(len(teams)), "v":np.zeros(len(teams)), "gp":np.zeros(len(teams)), "gc":np.zeros(len(teams)), "sg":np.zeros(len(teams))})
    ranking.set_index('time', inplace=True)
    
    for turn in turns: 
        if turn == 1:
            ranking_DF[season][turn] = classificar(matches.query("ano == " + str(season) + " and rodada == " + str(turn)), turn, ranking)
        else:
            ranking_DF[season][turn] = classificar(matches.query("ano == " + str(season) + " and rodada == " + str(turn)), turn, ranking_DF[season][turn-1])

In [224]:
def getPosicao(classificacao, row, tipo):
    return classificacao[row["ano"]][row["rodada"]].loc[row[tipo]]["posicao"]

In [225]:
matches["posicao_m"] = matches.apply(lambda row: getPosicao(ranking_DF, row, "mandante"),axis=1)

In [226]:
matches["posicao_v"] = matches.apply(lambda row: getPosicao(ranking_DF, row, "visitante"),axis=1)

In [227]:
matches.head(5)

,Unnamed: 0,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v
0,0,brasileiro-a,2018,9,04-06,Paraná,Fluminense,2,1,MANDANTE,19.0,8.0
1,1,brasileiro-a,2018,9,03-06,Ceará,Cruzeiro,0,1,VISITANTE,20.0,2.0
2,2,brasileiro-a,2018,9,03-06,América-MG,Atlético-PR,3,1,MANDANTE,10.0,16.0
3,3,brasileiro-a,2018,9,03-06,Bahia,Grêmio,0,2,VISITANTE,18.0,3.0
4,4,brasileiro-a,2018,9,03-06,Flamengo,Corinthians,1,0,MANDANTE,1.0,7.0


In [228]:
ranking_DF[2018][7]

,gc,gp,jogos,posicao,pts,sg,v
time,,,,,,,
Flamengo,6.0,13.0,7.0,1.0,14.0,7.0,4.0
Fluminense,7.0,11.0,7.0,2.0,13.0,4.0,4.0
Atlético-MG,7.0,9.0,7.0,3.0,13.0,2.0,4.0
São Paulo,6.0,10.0,7.0,4.0,13.0,4.0,3.0
Grêmio,3.0,8.0,7.0,5.0,12.0,5.0,3.0
Corinthians,6.0,10.0,7.0,6.0,11.0,4.0,3.0
Palmeiras,6.0,10.0,7.0,7.0,11.0,4.0,3.0
Sport Recife,10.0,9.0,7.0,8.0,11.0,-1.0,3.0
Cruzeiro,3.0,4.0,7.0,9.0,10.0,1.0,3.0


In [229]:
def aproveitamento5Rodadas (classificacao, row, tipo, jogos):    
    
    rodada = row["rodada"] - 1    
    rodada5r = rodada - jogos
    
    if rodada < 1:
        rodada = 1
        
    if rodada5r < 1:
        rodada5r = 1    
        
    pontos = classificacao[row["ano"]][rodada].loc[row[tipo]]["pts"]
    pontos5r = classificacao[row["ano"]][rodada5r].loc[row[tipo]]["pts"]
    
    if rodada == rodada5r:
        return 0        
    
    return round((pontos - pontos5r)/((rodada - rodada5r)*3), 2)

In [230]:
matches["aproveitamento_5r_v"] = matches.apply(lambda row: aproveitamento5Rodadas(ranking_DF, row, "visitante", 5),axis=1)

In [231]:
matches["aproveitamento_5r_m"] = matches.apply(lambda row: aproveitamento5Rodadas(ranking_DF, row, "mandante", 5),axis=1)

In [232]:
matches["aproveitamento_10r_v"] = matches.apply(lambda row: aproveitamento5Rodadas(ranking_DF, row, "visitante", 10),axis=1)

In [233]:
matches["aproveitamento_10r_m"] = matches.apply(lambda row: aproveitamento5Rodadas(ranking_DF, row, "mandante", 10),axis=1)

In [234]:
matches.head(5)

,Unnamed: 0,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,0,brasileiro-a,2018,9,04-06,Paraná,Fluminense,2,1,MANDANTE,19.0,8.0,0.67,0.20,0.67,0.14
1,1,brasileiro-a,2018,9,03-06,Ceará,Cruzeiro,0,1,VISITANTE,20.0,2.0,0.80,0.13,0.62,0.14
2,2,brasileiro-a,2018,9,03-06,América-MG,Atlético-PR,3,1,MANDANTE,10.0,16.0,0.27,0.27,0.29,0.33
3,3,brasileiro-a,2018,9,03-06,Bahia,Grêmio,0,2,VISITANTE,18.0,3.0,0.60,0.27,0.48,0.38
4,4,brasileiro-a,2018,9,03-06,Flamengo,Corinthians,1,0,MANDANTE,1.0,7.0,0.53,0.67,0.52,0.76


In [235]:
matches.to_csv(path_or_buf="data/matches-results-positions.9.csv",sep=",")